# Import packages

In [1]:
import pandas as pd
import numpy as np
import os
from scipy.optimize import minimize
from scipy.optimize import shgo

# Different isotherm models

In [3]:
def Arrh(T, dH, T_ref):
    exp_term = np.exp(np.abs(dH)/8.3145*(1/T-1/T_ref))
    return exp_term
    
def Lang(par,P):
    bP = par[1]*P
    deno = 1+ bP
    nume = par[0]*bP
    q = nume/deno
    return q

def Quad(par,P):
    bP = par[1]*P
    dPP = par[2]*P
    deno = 1+ bP + dPP
    nume = par[0]*(2*bP + dPP)
    q = nume/deno
    return q

def DSLa(par,P):
    nume1 = par[0]*par[1]*P
    deno1 = 1+par[1]*P
    nume2 = par[2]*par[3]*P
    deno2 = 1+par[3]*P
    q = nume1/deno1 + nume2/deno2
    return q    

iso_fn_candi = [Lang,Quad, DSLa]
iso_fn_candi_str = ['Lang','Quad', 'DSLa']
iso_par_num = [2,3,4]

# Objective Functions

In [4]:
def iso2err(par,P,q,iso_fn):
    par_arr = np.array(par)
    is_nega = par_arr<0
    penaltyy = np.sum(par_arr[is_nega]**2)*50
    par_arr[is_nega] = 0
    
    diff = iso_fn(par_arr,np.array(P)) - np.array(q)
    err_sum = np.sum(diff**2)
    return err_sum + penaltyy
# Test the iso2err
p_test = np.linspace(0,50, 51)
q_test = 3*0.1*p_test/(1+0.1*p_test)

err_test1 = iso2err([3,0.1], p_test,q_test, Lang)
err_test2 = iso2err([3,0.1,0.1], p_test,q_test, Quad)
err_test3 = iso2err([3,0.1,0.1,0.001], p_test,q_test, DSLa)

print(err_test1)
print(err_test2)
print(err_test3)

4.388038785291878e-30
116.55303488599569
0.00039860265674079767


# Fitting function

In [11]:
method_list = ['Nelder-mead','Powell','COBYLA','shgo']
def find_par(isofn, n_par, P,q, methods):
    p_arr = np.array(P)
    q_arr = np.array(q)
    obj_fn = lambda par: iso2err(par,p_arr,q_arr,isofn)
    optres_fun = []
    optres_x = []
    for me in methods: 
        if me =='shgo':
            bounds = np.zeros([n_par,2])
            bounds[:,1] = 5
            optres_tmp = shgo(obj_fn,bounds,)
        else:
            x0 = 2*np.ones(n_par)  # INITIAL GUESS !!!
            x0[0] = q[-1]
            optres_tmp = minimize(obj_fn,x0,method = me)
        optres_fun.append(optres_tmp.fun)
        optres_x.append(optres_tmp.x)
    bestm = np.argmin(optres_fun)
    par_sol = optres_x[bestm]
    fn_sol = optres_fun[bestm]
    return par_sol, fn_sol, optres_x, optres_fun

parsol_test, fnsol_test,_,_ = find_par(iso_fn_candi[0],iso_par_num[0],p_test,q_test, method_list)
#print(np.array(other_res).shape)
#print(other_res)
#print(optres_test[0])
#print(optres_test[1])
#print(np.argmin(optres_test[0]))

In [21]:
def best_isomodel(P,q):
    optfn_list = []
    optx_list = []
    for n_par,iso_fn in zip(iso_par_num, iso_fn_candi):
        parsol_tmp, fnsol_tmp, _,_ = find_par(iso_fn, n_par,P,q,method_list)
        optx_list.append(parsol_tmp)
        optfn_list.append(fnsol_tmp)
        if fnsol_tmp <= 4E-4:
            break
    print(optfn_list)
    argMIN = np.argmin(np.array(optfn_list))
    x_best = np.array(optx_list)[argMIN]
    iso_best = lambda pp: iso_fn_candi[argMIN](x_best, pp)
    str_best = iso_fn_candi_str[argMIN]
    fnval_best = np.array(optfn_list)[argMIN]
    return iso_best, x_best, str_best, fnval_best

isob_test, xb_test, strb_test, fvalb_test =best_isomodel(p_test,q_test)
print(isob_test)
print(xb_test)
print(strb_test)

[1.1469342501500302e-23]
<function best_isomodel.<locals>.<lambda> at 0x00000227A8910820>
[3.  0.1]
Lang
